# Собираем словари со-встречаемостей

Данный notebook содержит примеры команд и скриптов, с помощью которых можно собрать словари со-встречаемостей. Все необходимые для запуска файлы находятся в https://github.com/MichaelSolotky/bigartm-book/tree/cooc_dict/junk/cooc_dictionary. Просьба прочитать для начала разделы в документации по сборке файлов со-встречаемостей:
- http://docs.bigartm.org/en/stable/tutorials/bigartm_cli.html
- http://docs.bigartm.org/en/stable/tutorials/python_userguide/coherence.html

Предположим, мы склонировали репозиторий и находимся в дирректории с файлами. Посмотрим, что у нас есть:
```bash
ls -l
-rwxrwxrwx 1 root root     91 Jul  8 22:15 clean.sh
-rwxrwxrwx 1 root root  42383 Jul  8 22:32 cooc_df_
-rwxrwxrwx 1 root root  42383 Jul  8 22:32 cooc_tf_
-rwxrwxrwx 1 root root 163798 Jul  8 22:19 dictionary.txt
-rwxrwxrwx 1 root root  32617 Jul  8 22:34 Example_of_gathering.ipynb
-rwxrwxrwx 1 root root   2557 Jul  8 22:19 get_vocab.ipynb
-rwxrwxrwx 1 root root    199 Jul  8 22:32 launch.sh
-rwxrwxrwx 1 root root  46099 Jul  8 22:32 ppmi_df_
-rwxrwxrwx 1 root root  54793 Jul  8 22:32 ppmi_tf_
-rwxrwxrwx 1 root root  64380 Jul  8 22:19 vocab.txt
-rwxrwxrwx 1 root root  95266 May 10 20:22 vw.txt
```

- `vw.txt` $-$ коллекция в Vowpal Wabbit формате, в этом файле каждая документ записан в отдельной строке, первый токен в строке - название документа, далее в произвольных местах могут находиться токены, начинающиеся с символа '|', они обозначают модальность. Все последующие токены в документе (которые не начинаются с '|') относятся к этой модальности. Дефолтная модальность обозначается |@default_class. Если в документе не была указана модальность, то считается, что токены относятся к дефолтной модальности;
- `vocab.txt` $-$ словарь валидных токенов, со-встречаемости которых будут подсчитаны. Пока что со-встречаемости считаются только внутри модальностей, то есть если 2 токена из разных модальностей, такие со-встречаемости учитываться не будут. Если токен не принадлежит дефолтной модальности (т.е. @default_class), его модальность необходимо указать в `vocab.txt`. Стоит обратить внимание, что в vocab модальность указывется без символа '|', к примеру, @default_class;
- ноутбук `get_vocab.ipynb` позволяет получить `vocab.txt` по `vw.txt`, в него записываются все уникальные токены коллекции (за исключением токенов модальностей). Обратите внимание, ноутбук пишет `vocab.txt` и `dictionary.txt` в папку `batches`;
- `dictionary.txt` $-$ artm dictionary, собранный по `vw.txt`, побочный результат работы `get_vocab.ipynb`, в принципе может пригодиться для дальнейших экспериментов с коллекцией;
- скрипт `clean.sh` производит очистку текущей дирректории от промежуточных файлов (необязателен, но с ним удобнее);
- файлы `cooc_tf_`, `cooc_df_`, `ppmi_tf_`, `ppmi_df_` $-$ файлы со-встречаемостей и посчитанные по ним PPMI, результат работы алгоритма;
- `launch.sh` $-$ скрипт для запуска утилиты подсчёта со-встречаемостей и PPMI.

Заглянем внутрь `launch.sh`:
```bash
#! /bin/bash

bigartm -c vw.txt -v vocab.txt --cooc-window 10 --cooc-min-tf 5 --write-cooc-tf cooc_tf_
--cooc-min-df 5 --write-cooc-df cooc_df_ --write-ppmi-tf ppmi_tf_ --write-ppmi-df 
ppmi_df_
```

Что данные ключи значат, по порядку:
- `-c` $-$ прочитать коллекцию;
- `-v` $-$ прочитать vocab;
- `--cooc-window` $-$ задать ширину окна (со-встречаемость токенов учитывается только внутри некоторого окна);
- `--cooc-min-tf` $-$ задать минимальное значение cooc TF (значение ниже данного порога не будут писаться в выходной файл);
- `--cooc-min-df` $-$ аналогично предыдущему, только для cooc DF;
- `--write-cooc-tf` $-$ записать файл с cooc TF по заданному пути, далее указывается путь;
- `--write-cooc-df`, `--write-ppmi-tf`, `--write-ppmi-df` $-$ аналогично для файлов cooc DF, PPMI TF, PPMI DF.

Сейчас те, кто не прочитал документацию, могут не понять, что такое cooc TF, какие-то PPMI, дублируем:

#### positive PMI: $PPMI(u,v)=\left[\log\cfrac{n(u,v)n}{n(u)n(v)}\right]_{+}$;

#### Cooc TF: $n(u, v) = \sum_{d = 1}^{|D|}$ $\sum_{i = 1}^{N_d}$ $\sum_{j = 1}^{N_d}$ $[0 < |i - j| \leq k]$ $[w_{di} = u]$ $[w_{dj} = v]$;

#### Cooc DF: $n(u, v) = \sum_{d = 1}^{|D|} [\, \exists \, i, j : w_{di} = u, w_{dj} = v, 0 < |i - j| \leq k]$.

По сути cooc TF означает сколько раз пара встретилась в коллекции внутри окна заданной ширины, а cooc DF $-$ в скольких документах она встретилась хотя бы раз в окне.

В данном примере коллекция взята довольно маленькая $-$ просто, чтоб показать, как всё работает.
Конечно, по таким маленьким коллекциях оценивать близость пар слов - не самая лучшая идея, их надо оценивать на больших коллекциях типа Википедии, на которых дисперсия оценки вероятностей относительно маленькая.

## Часто задаваемые вопросы и ошибки

- Почему утилита отработала, а никакие файлы не создались. **Ответ**: такое может быть только в случае, если утилита не нашла ни 1 со-встречаемости. Возожно, в vocab неправильно указаны модальности (или не указаны вовсе, но в тексте коллекции все слова в недефолтной модальности) или выбран слишком высокий порог, так что со-встречаемость всех пар оказалась ниже порога.